<a href="https://colab.research.google.com/github/hkvision/bigdl-demo/blob/main/lstm_pollution_dllib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install jdk8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
# Set environment variable JAVA_HOME.
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)


In [2]:
!pip install --pre --upgrade bigdl-dllib-spark3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.5 MB 1.4 MB/s 
     |████████████████████████████████| 51.0 MB 36 kB/s 
     |████████████████████████████████| 212.4 MB 72 kB/s 
     |████████████████████████████████| 198 kB 56.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=27d59f9971573e9891068ad880c6fff4341b117a01cfda9c114a1c72c290498b
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv

--2022-07-25 11:07:02--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2010492 (1.9M) [text/plain]
Saving to: ‘pollution.csv’

pollution.csv       100%[===================>]   1.92M  --.-KB/s    in 0.04s   

2022-07-25 11:07:03 (47.3 MB/s) - ‘pollution.csv’ saved [2010492/2010492]



In [4]:
import numpy as np
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [6]:
# load dataset
dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values
dataset.head()

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
No,,,,,,,,,,,,
1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [7]:
# integer encode direction
encoder = LabelEncoder()
values[:, -4] = encoder.fit_transform(values[:, -4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [9]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [13]:
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var3(t),var4(t),var5(t),var6(t),var7(t),var8(t),var9(t),var10(t),var11(t),var12(t)
25,0.0,0.0,0.033333,0.000000,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,...,0.033333,0.043478,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0
26,0.0,0.0,0.033333,0.043478,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,...,0.033333,0.086957,0.159960,0.426471,0.229508,0.545454,0.666667,0.005332,0.000000,0.0
27,0.0,0.0,0.033333,0.086957,0.159960,0.426471,0.229508,0.545454,0.666667,0.005332,...,0.033333,0.130435,0.182093,0.485294,0.229508,0.563637,0.666667,0.008391,0.037037,0.0
28,0.0,0.0,0.033333,0.130435,0.182093,0.485294,0.229508,0.563637,0.666667,0.008391,...,0.033333,0.173913,0.138833,0.485294,0.229508,0.563637,0.666667,0.009912,0.074074,0.0
29,0.0,0.0,0.033333,0.173913,0.138833,0.485294,0.229508,0.563637,0.666667,0.009912,...,0.033333,0.217391,0.109658,0.485294,0.213115,0.563637,0.666667,0.011433,0.111111,0.0


In [14]:
# drop columns we don't want to predict
reframed.drop(reframed.columns[[9, 10, 11, 12, 13, 14, 15]], axis=1, inplace=True)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var5(t),var6(t),var7(t),var8(t),var9(t),var10(t),var11(t),var12(t)
25,0.0,0.0,0.033333,0.000000,0.129779,0.352941,0.245902,0.527273,0.666667,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0
26,0.0,0.0,0.033333,0.043478,0.148893,0.367647,0.245902,0.527273,0.666667,0.159960,0.426471,0.229508,0.545454,0.666667,0.005332,0.000000,0.0
27,0.0,0.0,0.033333,0.086957,0.159960,0.426471,0.229508,0.545454,0.666667,0.182093,0.485294,0.229508,0.563637,0.666667,0.008391,0.037037,0.0
28,0.0,0.0,0.033333,0.130435,0.182093,0.485294,0.229508,0.563637,0.666667,0.138833,0.485294,0.229508,0.563637,0.666667,0.009912,0.074074,0.0
29,0.0,0.0,0.033333,0.173913,0.138833,0.485294,0.229508,0.563637,0.666667,0.109658,0.485294,0.213115,0.563637,0.666667,0.011433,0.111111,0.0


In [15]:
# split into train and test sets
values = reframed.values
n_train_hours = 365 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(8760, 1, 16) (8760,) (32783, 1, 16) (32783,)


In [16]:
# Change the imports to DLlib keras-like api
from bigdl.dllib.nncontext import init_nncontext
from bigdl.dllib.keras.models import Sequential
from bigdl.dllib.keras.layers import LSTM, Dense
from bigdl.dllib.nncontext import ZooContext

ZooContext.log_output = True # (this will display terminal's stdout and stderr in the Jupyter notebook).

# initialize BigDL
sc = init_nncontext(cluster_mode="local")

Current pyspark location is : /usr/local/lib/python3.7/dist-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /usr/local/lib/python3.7/dist-packages/bigdl/share/core/lib/all-2.1.0-20220314.094552-2.jar:/usr/local/lib/python3.7/dist-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_3.1.2-2.1.0-SNAPSHOT-jar-with-dependencies.jar pyspark-shell 
Successfully got a SparkContext


In [17]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
model.fit(train_X, train_y, nb_epoch=50, batch_size=72, validation_data=(test_X, test_y))

Streaming output truncated to the last 5000 lines.
2022-07-25 11:33:32.297 [Thread-4] INFO  com.intel.analytics.bigdl.dllib.optim.DistriOptimizer$ - [Epoch 11 2952/8760][Iteration 1261][Wall Clock 84.781220148s] Trained 72.0 records in 0.033460819 seconds. Throughput is 2151.7705 records/second. Loss is 0.0036913669. 
2022-07-25 11:33:32.332 [Thread-4] INFO  com.intel.analytics.bigdl.dllib.optim.DistriOptimizer$ - [Epoch 11 3024/8760][Iteration 1262][Wall Clock 84.815558014s] Trained 72.0 records in 0.034337866 seconds. Throughput is 2096.8105 records/second. Loss is 0.002175885. 
2022-07-25 11:33:32.379 [Thread-4] INFO  com.intel.analytics.bigdl.dllib.optim.DistriOptimizer$ - [Epoch 11 3096/8760][Iteration 1263][Wall Clock 84.861971062s] Trained 72.0 records in 0.046413048 seconds. Throughput is 1551.2878 records/second. Loss is 0.009455023. 
2022-07-25 11:33:32.410 [Thread-4] INFO  com.intel.analytics.bigdl.dllib.optim.DistriOptimizer$ - [Epoch 11 3168/8760][Iteration 1264][Wall Cloc

In [18]:
from math import sqrt
from sklearn.metrics import mean_squared_error

# make a prediction
# will return an RDD and we collect them to form the result ndarray
yhat = model.predict(test_X).collect()
yhat = np.array([y[0] for y in yhat])
# calculate Root Mean Squared Error (RMSE)
rmse = sqrt(mean_squared_error(test_y, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.037
